In [1]:
config = {
    "pretrained_model": "bert-base-cased",
    "tokenizer": "bert-base-cased",
    "max_seq_length": 512,
    "batch_size": 10,
    "lr": 2e-5,
    "epochs": 10,
    "device": "cuda",
    "gpu_ids": "0,1,2,3",
    "seed": 2020,
    "fp16": False,
    "loss_scale": 0,
    "gradient_accumulation_steps":1,
    "warmup_proportion": 0.1,
    "gradient_accumulation_steps": 1,
    "num_labels": 4,
    "is_multilabel": False,
    "valid_metric": "macro_f1",
    "model_save_dir": "../checkpoints/bert_cased_512_biocaster_4cate_20200903/",
    "patience": 4,
}

In [2]:
import sys
sys.path.append("../")
from beta_nlp.utils.data_util import biocaster2df
data_file = "/home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml"
data_df = biocaster2df(data_file)
data_df["source"] = "Biocaster"
data_df.head(3)

parse biocaser data from /home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml, docs number:1003, lablels number:1003


,docs,labels,source
0,\nBird Flu Outbreak Drill Spooks Manitoba Town...,0,Biocaster
1,\nTyphoid outbreak in Agusan del Sur town unde...,3,Biocaster
2,\n Typhoid Outbreak In Central Nepal November...,3,Biocaster


In [3]:
from sklearn.utils import shuffle
data_df = shuffle(data_df).reset_index()
data_df["flag"]=None
train_index = int(len(data_df.index)*0.8)
valid_index = train_index + round(len(data_df.index)*0.1)
data_df.iloc[:train_index]["flag"]="train"
data_df.iloc[train_index:valid_index]["flag"]="valid"
data_df.iloc[valid_index:]["flag"]="test"

<ipython-input-3-79406d09754d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[:train_index]["flag"]="train"
<ipython-input-3-79406d09754d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[train_index:valid_index]["flag"]="valid"
<ipython-input-3-79406d09754d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [4]:
train_set = data_df[data_df["flag"]=="train"]
dev_set = data_df[data_df["flag"]=="valid"]
test_set = data_df[data_df["flag"]=="test"]

In [5]:
len(train_set.index),len(dev_set.index),len(test_set.index)

(802, 100, 101)

In [6]:
from beta_nlp.models.bert_cls import BertModel
cls = BertModel(config)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [7]:
len(train_set.index),len(dev_set.index),len(test_set.index)

(802, 100, 101)

In [8]:
cls.train(train_set,dev_set)


Execute [train_an_epoch] method costing 31637.72 ms
[Epoch 0] loss: 89.48777252435684



Execute [eval] method costing 1350.36 ms
--------------------------------------------------------------------------------
[Epoch 0]performance on validation set
+----+-----------------+----------+
|    | metrics         |   values |
|----+-----------------+----------|
|  0 | accuracy        | 0.75     |
|  1 | macro_precision | 0.356739 |
|  2 | macro_recall    | 0.395833 |
|  3 | macro_f1        | 0.375266 |
|  4 | micro_precision | 0.75     |
|  5 | micro_recall    | 0.75     |
|  6 | micro_f1        | 0.75     |
+----+-----------------+----------+
--------------------------------------------------------------------------------


/home/zm324/anaconda3/envs/doc_cls/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Execute [train_an_epoch] method costing 32368.51 ms
[Epoch 1] loss: 52.58577781915665



Execute [eval] method costing 1377.92 ms
--------------------------------------------------------------------------------
[Epoch 1]performance on validation set
+----+-----------------+----------+
|    | metrics         |   values |
|----+-----------------+----------|
|  0 | accuracy        | 0.81     |
|  1 | macro_precision | 0.394505 |
|  2 | macro_recall    | 0.4375   |
|  3 | macro_f1        | 0.414886 |
|  4 | micro_precision | 0.81     |
|  5 | micro_recall    | 0.81     |
|  6 | micro_f1        | 0.81     |
+----+-----------------+----------+
--------------------------------------------------------------------------------


...


Execute [eval] method costing 1445.48 ms
--------------------------------------------------------------------------------
[Epoch 7]performance on validation set
+----+-----------------+----------+
|    | metrics         |   values |
|----+-----------------+----------|
|  0 | accuracy        | 0.87     |
|  1 | macro_precision | 0.671711 |
|  2 | macro_recall    | 0.533333 |
|  3 | macro_f1        | 0.543382 |
|  4 | micro_precision | 0.87     |
|  5 | micro_recall    | 0.87     |
|  6 | micro_f1        | 0.87     |
+----+-----------------+----------+
--------------------------------------------------------------------------------



Execute [train_an_epoch] method costing 34411.60 ms
[Epoch 8] loss: 10.491857270943001



Execute [eval] method costing 1448.87 ms



Execute [train_an_epoch] method costing 34361.96 ms
[Epoch 9] loss: 10.587603650230449



Execute [eval] method costing 1439.04 ms



In [9]:
cls.test(test_set)


Execute [eval] method costing 1453.96 ms


{'accuracy': 0.8118811881188119,
 'macro_precision': 0.4544794188861985,
 'macro_recall': 0.5133538777606574,
 'macro_f1': 0.4742273180458624,
 'micro_precision': 0.8118811881188119,
 'micro_recall': 0.8118811881188119,
 'micro_f1': 0.8118811881188119}

In [10]:
test_set.iloc[1:10]

,index,docs,labels,source,flag
903,263,\n\nSource: University Of Toronto \nDate: Marc...,0,Biocaster,test
904,554,\n Resident struck by Legionnaires Health off...,3,Biocaster,test
905,703,\n Energy costs to drive inflation \n \n \n Hi...,0,Biocaster,test
906,658,\nNYT-01-18-96 1853EST\nBC-SATELLITE-TV-NYT CA...,0,Biocaster,test
907,860,\n China says time right for Taiwan talks .\n...,0,Biocaster,test
908,780,\n West Nile Found In Blood Donation\nWASHING...,3,Biocaster,test
909,134,\n SARS CRISIS: Eighth victim of virus to go ...,0,Biocaster,test
910,801,\n HFMD death toll rises to 9 in eastern Mala...,3,Biocaster,test
911,365,\n TB Cases Increase in Mpigi District \nThe ...,3,Biocaster,test


In [11]:
cls.predict(test_set.iloc[1:10])


Execute [predict] method costing 156.87 ms


array([0, 3, 0, 0, 0, 0, 3, 3, 0])